In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from scipy.signal import savgol_filter

# Load data
filename = '../npz/BTCUSD_1T.npz'
data = np.load(filename, allow_pickle=True)['data']
loaded_df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])

# Convert 'timestamp' to datetime and set it as the index
loaded_df['timestamp'] = pd.to_datetime(loaded_df['timestamp'])
loaded_df.set_index('timestamp', inplace=True)

# Convert all price and volume columns to numeric format
for column in ['open', 'high', 'low', 'close', 'volume']:
    loaded_df[column] = pd.to_numeric(loaded_df[column], errors='coerce')

In [ ]:
df = loaded_df[3000:3200]

In [ ]:
# Simple Moving Average (SMA)
window_size = 10  # размер окна для усреднения
sma10 = df['close'].rolling(window=window_size).mean()

# Визуализация
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['close'], label="Original", alpha=0.5)
plt.plot(df.index, sma10, label=f"SMA ({window_size})", linewidth=2)
plt.legend()
plt.title("Simple Moving Average (SMA)")
plt.show()


In [ ]:
# Exponential Moving Average (EMA)
span = 10  # Период для усреднения
ema10 = df['close'].ewm(span=span, adjust=False).mean()

# Визуализация
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['close'], label="Original", alpha=0.5)
plt.plot(df.index, ema10, label=f"EMA ({span})", linewidth=2)
plt.legend()
plt.title("Exponential Moving Average (EMA)")
plt.show()


In [ ]:
from scipy.signal import medfilt

# Median Filter
kernel_size = 5  # Размер ядра должен быть нечётным
med_filtered = medfilt(df['close'], kernel_size=kernel_size)

# Визуализация
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['close'], label="Original", alpha=0.5)
plt.plot(df.index, med_filtered, label=f"Median Filter (k={kernel_size})", linewidth=2)
plt.legend()
plt.title("Median Filter")
plt.show()


In [ ]:
# Savitzky-Golay Filter
window_length = 15  # Должен быть нечётным
polyorder = 3  # Степень полинома
savgol_filtered = savgol_filter(df['close'], window_length=window_length, polyorder=polyorder)

# Визуализация
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['close'], label="Original", alpha=0.5)
plt.plot(df.index, savgol_filtered, label=f"Savitzky-Golay (w={window_length}, p={polyorder})", linewidth=2)
plt.legend()
plt.title("Savitzky-Golay Filter")
plt.show()


In [ ]:
from scipy.signal import savgol_filter
import numpy as np

# Параметры Savitzky-Golay
window_length = 15  # Должен быть нечётным
polyorder = 3

# Количество точек, которые он не сможет обработать на конце
missing_points = window_length // 2

# Экстраполяция полиномом для заполнения краёв
def extend_series(series, extend_by):
    x = np.arange(len(series))
    y = series.values
    coeffs = np.polyfit(x[-window_length:], y[-window_length:], polyorder)  # Подгоняем полином
    poly = np.poly1d(coeffs)
    
    # Генерируем дополнительные точки
    extra_x = np.arange(len(series), len(series) + extend_by)
    extra_y = poly(extra_x)
    
    # Соединяем старые и новые данные
    extended_series = np.concatenate([y, extra_y])
    return extended_series

# Дополним данные перед фильтрацией
extended_close = extend_series(df['close'], missing_points)

# Применяем Savitzky-Golay
savgol_smoothed = savgol_filter(extended_close, window_length=window_length, polyorder=polyorder)

# Обрезаем дополненные точки
savgol_smoothed = savgol_smoothed[:len(df)]

# Визуализация
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['close'], label="Original", alpha=0.5)
plt.plot(df.index, savgol_smoothed, label="Savitzky-Golay (Extended)", linewidth=2)
plt.plot(df.index, savgol_filtered, label=f"Savitzky-Golay (w={window_length}, p={polyorder})", linewidth=2)
plt.legend()
plt.title("Savitzky-Golay with Polynomial Extrapolation")
plt.show()


In [ ]:
import pywt

# Wavelet Denoising
def wavelet_denoise(data, wavelet='db4', level=2):
    coeffs = pywt.wavedec(data, wavelet, level=level)
    coeffs[1:] = [pywt.threshold(c, value=np.std(c), mode="soft") for c in coeffs[1:]]
    return pywt.waverec(coeffs, wavelet)[:len(data)]  # Ограничиваем длину

wavelet_smoothed = wavelet_denoise(df['close'])

# Визуализация
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['close'], label="Original", alpha=0.5)
plt.plot(df.index, wavelet_smoothed, label="Wavelet Smoothed", linewidth=2)
plt.legend()
plt.title("Wavelet Denoising")
plt.show()


In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Holt-Winters Exponential Smoothing
hw_model = ExponentialSmoothing(df['close'], trend='add', seasonal=None)
hw_fitted = hw_model.fit().fittedvalues

# Визуализация
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['close'], label="Original", alpha=0.5)
plt.plot(df.index, hw_fitted, label="Holt-Winters", linewidth=2)
plt.legend()
plt.title("Holt-Winters Exponential Smoothing")
plt.show()
